In [6]:
import pandas as pd
filepath = "https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

dfs = pd.read_excel(filepath, sheet_name=None)

#dfs.keys()
dfs['header'].head()

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
0,566910,2,27/03/2008,WROM,D051910639385,NaN,NaN
1,384923,2,09/09/2011,REFU,L034811502612,NaN,NaN
2,709901,1,14/02/2013,ABAN,Y021751958510,NaN,NaN
3,550084,2,04/05/2008,AOTH,R095578985099,NaN,NaN
4,710229,2,15/04/2002,REFU,Z095096287583,NaN,NaN


In [8]:
header = dfs['header']

# ssda903 102 - date of birth is not a valid date

# coercing errors returns NATs for invalid dates
# if I slice out all NaTs - I have all the invalid dates!
header['DOB_dt'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')

# .isna() rewutnrs true where the value in a row in a column is an Na
invalid_dobs = header['DOB_dt'].isna()
error_rows = header[invalid_dobs]

error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [9]:
# .isin() -> returns true where a column has a value in a list... it is in the list
# 103 - The ethnicity code is either not valid or has not been entered.

ethncity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]
# ~ 
error_rows = header[~header['ETHNIC'].isin(ethncity_codes)]
error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [15]:
ad1 = dfs['ad1']
foster_codes = [0, 1]
error_rows = ad1[~ad1['FOSTER_CARE'].isin(foster_codes)]
error_rows

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [24]:
p4a = dfs['placed_for_adoption']
# 115 - Date of Local Authority's (LA) decision that a child 
# should be placed for adoption is not a valid date."

# select the placed for adoption table from dfs, 
# convert the correct column to a datetime, 
# make the slice

adoption = dfs['placed_for_adoption']

adoption['DATE_PLACED_dt'] = pd.to_datetime(adoption['DATE_PLACED'], format="%d/%m/%Y", errors='coerce')




In [25]:
# 184 - Date of decision that a child should be placed for adoption is before the child was born.
# return rows where DOB after DATE_PLACED -> DOB > DATE_PLACED

merged_df = p4a.merge(header[['CHILD', 'DOB_dt']], on='CHILD', how='left', suffixes=[None, '_head'])

error_rows = merged_df[merged_df["DOB_dt"] > merged_df['DATE_PLACED_dt']]

error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_dt,DOB_dt
